In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

In [3]:
import os
from langchain_community.utilities import SQLDatabase

# MySQL 연결 설정
# .env 파일에 다음 변수들을 설정하세요:
# DB_USER=your_username
# DB_PASSWORD=your_password
# DB_HOST=localhost
# DB_PORT=3306
# DB_NAME=your_database

db_uri = f"mysql+pymysql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"

try:
    db = SQLDatabase.from_uri(db_uri)
    print(f"Dialect: {db.dialect}")
    print(f"Available tables: {db.get_usable_table_names()}")
    
    # 테이블이 있다면 샘플 데이터 조회
    tables = db.get_usable_table_names()
    if tables:
        sample_table = tables[0]
        print(f'Sample output from {sample_table}: {db.run(f"SELECT * FROM {sample_table} LIMIT 5;")}')
    else:
        print("No tables found in the database")
        
except Exception as e:
    print(f"Database connection error: {e}")
    print("Please check your database credentials in .env file")

Dialect: mysql
Available tables: ['actor', 'address', 'category', 'city', 'country', 'customer', 'film', 'film_actor', 'film_category', 'film_text', 'inventory', 'language', 'payment', 'rental', 'staff', 'store']
Sample output from actor: [(1, 'PENELOPE', 'GUINESS', datetime.datetime(2006, 2, 15, 4, 34, 33)), (2, 'NICK', 'WAHLBERG', datetime.datetime(2006, 2, 15, 4, 34, 33)), (3, 'ED', 'CHASE', datetime.datetime(2006, 2, 15, 4, 34, 33)), (4, 'JENNIFER', 'DAVIS', datetime.datetime(2006, 2, 15, 4, 34, 33)), (5, 'JOHNNY', 'LOLLOBRIGIDA', datetime.datetime(2006, 2, 15, 4, 34, 33))]


c:\Users\dandycode\Documents\GitHub\langgraph-sql-agent\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'geometry' of column 'location'
  self._metadata.reflect(


In [4]:
from langchain_community.agent_toolkits import create_sql_agent

# SQL Agent 생성 (데이터베이스 연결이 성공한 경우에만)
if 'db' in locals():
    agent_executor = create_sql_agent(
        llm=llm,
        db=db,
        agent_type="openai-tools",
        verbose=True
    )
    print("SQL Agent created successfully!")
else:
    print("Database connection required to create SQL Agent")

SQL Agent created successfully!


In [5]:
# # SQL Agent 테스트 쿼리
# if 'agent_executor' in locals():
#     # 예시 질문들
#     test_questions = [
#         "데이터베이스에 어떤 테이블들이 있나요?",
#         "각 테이블의 스키마를 보여주세요",
#         # "가장 많은 레코드를 가진 테이블은 무엇인가요?"
#     ]
    
#     for question in test_questions:
#         print(f"\n질문: {question}")
#         try:
#             result = agent_executor.invoke({"input": question})
#             print(f"답변: {result['output']}")
#         except Exception as e:
#             print(f"오류: {e}")
#         print("-" * 50)
# else:
#     print("SQL Agent가 생성되지 않았습니다. 데이터베이스 연결을 확인하세요.")

In [6]:
# 대화형 SQL Agent - SQL 쿼리도 함께 출력
import re
from io import StringIO
import sys

if 'agent_executor' in locals():
    print("=" * 60)
    print("🤖 SQL Agent와 대화하기 (SQL 쿼리 포함)")
    print("=" * 60)
    print("💡 예시 질문들:")
    print("   - 영화 테이블에서 가장 인기 있는 영화 5개를 보여주세요.")
    print("   - 각 카테고리별 영화 수를 알려주세요.")
    print("   - 가장 많이 대여된 영화는 무엇인가요?")
    print("   - 고객별 총 결제 금액을 보여주세요")
    print("   - 가장 높은 총 매출을 기록한 영화 카테고리 5개와 그 매출액을 내림차순으로 알려주세요.")
    print("   - 현재 재고 목록에 있지만, 한 번도 대여된 적이 없는 영화의 제목과 해당 영화의 재고 개수를 조회해 주세요.")

    print("   - 'quit' 또는 'exit'를 입력하면 종료됩니다")
    print("-" * 60)
    
    while True:
        try:
            # 사용자 입력 받기
            user_question = input("\n❓ 질문을 입력하세요: ").strip()
            
            # 종료 조건
            if user_question.lower() in ['quit', 'exit', '종료', 'q']:
                print("👋 SQL Agent 대화를 종료합니다.")
                break
            
            # 빈 입력 처리
            if not user_question:
                print("⚠️  질문을 입력해주세요.")
                continue
            
            print(f"\n🔍 질문: {user_question}")
            print("🤔 생각 중...")
            
            # stdout 캡처를 위한 설정
            old_stdout = sys.stdout
            captured_output = StringIO()
            sys.stdout = captured_output
            
            try:
                # SQL Agent 실행 (verbose=True이므로 SQL 쿼리가 출력됨)
                result = agent_executor.invoke({"input": user_question})
            finally:
                # stdout 복원
                sys.stdout = old_stdout
            
            # 캡처된 출력에서 SQL 쿼리 추출
            captured_text = captured_output.getvalue()
            
            # SQL 쿼리 패턴 찾기
            sql_pattern = r"Invoking: `sql_db_query` with `\{'query': '([^']+)'\}`"
            sql_matches = re.findall(sql_pattern, captured_text, re.DOTALL)
            sql_queries = [match for match in sql_matches if match.strip().upper().startswith('SELECT')]
            
            # SQL 쿼리 출력
            if sql_queries:
                print(f"\n📝 실행된 SQL 쿼리:")
                for i, query in enumerate(sql_queries, 1):
                    clean_query = query.strip().replace('\n', ' ')
                    print(f"   {i}. {clean_query}")
            
            print(f"\n✅ 답변:")
            print(result['output'])
            print("\n" + "="*60)
            
        except KeyboardInterrupt:
            print("\n\n👋 사용자가 중단했습니다. SQL Agent 대화를 종료합니다.")
            break
        except Exception as e:
            print(f"\n❌ 오류가 발생했습니다: {e}")
            print("다시 시도해주세요.\n")
            
else:
    print("❌ SQL Agent가 생성되지 않았습니다. 위의 셀들을 먼저 실행해주세요.")

🤖 SQL Agent와 대화하기 (SQL 쿼리 포함)
💡 예시 질문들:
   - 영화 테이블에서 가장 인기 있는 영화 5개를 보여주세요
   - 각 카테고리별 영화 수를 알려주세요
   - 가장 많이 대여된 영화는 무엇인가요?
   - 고객별 총 결제 금액을 보여주세요
   - 가장 높은 총 매출을 기록한 영화 카테고리 5개와 그 매출액을 내림차순으로 알려주세요.
   - 현재 재고 목록에 있지만, 한 번도 대여된 적이 없는 영화의 제목과 해당 영화의 재고 개수를 조회해 주세요.
   - 'quit' 또는 'exit'를 입력하면 종료됩니다
------------------------------------------------------------

🔍 질문: 각 카테고리별 영화 수를 알려주세요
🤔 생각 중...


c:\Users\dandycode\Documents\GitHub\langgraph-sql-agent\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:363: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\dandycode\Documents\GitHub\langgraph-sql-agent\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:375: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables



📝 실행된 SQL 쿼리:
   1. SELECT c.name, COUNT(fc.film_id) AS film_count FROM category c JOIN film_category fc ON c.category_id = fc.category_id GROUP BY c.name ORDER BY film_count DESC;

✅ 답변:
각 카테고리별 영화 수는 다음과 같습니다:

*   Sports: 74
*   Foreign: 73
*   Family: 69
*   Documentary: 68
*   Animation: 66
*   Action: 64
*   New: 63
*   Drama: 62
*   Games: 61
*   Sci-Fi: 61
*   Children: 60
*   Comedy: 58
*   Classics: 57
*   Travel: 57
*   Horror: 56
*   Music: 51

👋 SQL Agent 대화를 종료합니다.
